In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import json
import re
import time as t
from typing import Tuple, List, Dict, Any, Optional

# ====== Settings (edit these two as needed) ======
input_dir = Path("/Users/timtoepper/Downloads/Banks_Assignment_Tim")   # folder with CSVs
output_dir = input_dir / "sampled_outputs"                    # where to save sampled files
# =================================================

output_dir.mkdir(parents=True, exist_ok=True)

# For simple time measurements in logs
start = t.time()

# Sampling parameters
TOTAL_RECORDS = 10_000
FIXED_RECORDS = 1_000
BLOCK_SIZE = 100
NUM_RANDOM_BLOCKS = (TOTAL_RECORDS - FIXED_RECORDS) // BLOCK_SIZE  # 90 blocks

RNG = np.random.default_rng(42)  # reproducible sampling

# ────── Processing Functions ────────────────────────────────────────────────────
def parse_price(j: Any) -> Dict[str, Any]:
    """Parse a JSON string from price_overview; return normalized fields."""
    if pd.isna(j):
        return {
            "price_final_cents": None,
            "price_initial_cents": None,
            "currency": None,
            "discount_percent": None,
            "price_final": None,
            "price_initial": None,
            "price_final_formatted": None,
        }
    try:
        obj = json.loads(j)
        return {
            "price_final_cents": obj.get("final"),
            "price_initial_cents": obj.get("initial"),
            "currency": obj.get("currency"),
            "discount_percent": obj.get("discount_percent"),
            "price_final": (obj.get("final") or 0) / 100.0,
            "price_initial": (obj.get("initial") or 0) / 100.0,
            "price_final_formatted": obj.get("final_formatted"),
        }
    except Exception:
        # Fallback: return empty fields if a row is malformed
        return {
            "price_final_cents": None,
            "price_initial_cents": None,
            "currency": None,
            "discount_percent": None,
            "price_final": None,
            "price_initial": None,
            "price_final_formatted": None,
        }

def clean_languages(raw: Any, TAG_RE: re.Pattern) -> Tuple[List[str], List[str]]:
    """
    Remove HTML tags, split by comma, normalize names, and detect languages
    with full audio support (marked by a trailing '*').
    Returns (all_languages, full_audio_subset).
    """
    if pd.isna(raw):
        return [], []
    # Remove <br> and other tags, keep the "*" that indicates full audio support
    txt = TAG_RE.sub("", str(raw))
    # Now split on commas and normalize
    langs = [p.strip() for p in txt.split(",") if p.strip()]
    # Items may look like "English*" where * = full audio support
    full_audio = [l.replace("*", "").strip() for l in langs if l.endswith("*")]
    normal = [l.replace("*", "").strip() for l in langs]
    return normal, full_audio

def time_helper(text: str) -> None:
    print(f"── {text}   Time used: {int((t.time()-start)//60)}m  {round((t.time()-start)%60, 2)}s")

def read_games_csv(input_dir: Path) -> Optional[pd.DataFrame]:
    """Try to read a games CSV from common filenames; return None if not found."""
    candidates = [
        input_dir / "games_cleaned.csv",
        input_dir / "steam_games.csv",
        input_dir / "Games_cleaned.csv",
    ]
    if not any(p.exists() for p in candidates):
        # try globbing as a fallback
        globs = list(input_dir.glob("games*.csv")) + list(input_dir.glob("*games*.csv"))
        candidates.extend(globs)

    for p in candidates:
        if isinstance(p, Path) and p.exists():
            try:
                df = pd.read_csv(
                    p,
                    engine="python",
                    sep=",",
                    quotechar='"',
                    escapechar="\\",
                    on_bad_lines="skip",
                )
                print(f"Loaded: {p.name} | rows={len(df):,} cols={len(df.columns)}")
                return df
            except Exception as e:
                print(f"Warning: failed to read {p}: {e}")
    return None

def expand_price_overview(df: pd.DataFrame) -> pd.DataFrame:
    """If 'price_overview' exists, expand its JSON into columns."""
    if "price_overview" not in df.columns:
        print("Note: 'price_overview' column not found; skipping JSON expansion.")
        return df
    price_expanded = df["price_overview"].apply(parse_price).apply(pd.Series)
    # Avoid duplicate columns if some already exist with same names
    price_expanded = price_expanded[[c for c in price_expanded.columns if c not in df.columns]]
    df_out = pd.concat([df.drop(columns=["price_overview"]), price_expanded], axis=1)
    return df_out

def expand_languages(df: pd.DataFrame) -> pd.DataFrame:
    """If 'languages' exists, clean it and add two list-typed columns."""
    if "languages" not in df.columns:
        print("Note: 'languages' column not found; skipping language cleaning.")
        return df
    TAG_RE = re.compile(r"<.*?>")
    langs_parsed = df["languages"].apply(lambda x: clean_languages(x, TAG_RE))
    df["languages_clean"] = langs_parsed.apply(lambda t: t[0]).astype(object)
    df["languages_full_audio"] = langs_parsed.apply(lambda t: t[1]).astype(object)
    # keep or drop original column depending on your preference
    df = df.drop(columns=["languages"])
    return df

def to_csv(df: pd.DataFrame, path: Path) -> None:
    """Write DataFrame to CSV with safe options for large text."""
    df.to_csv(path, index=False, quoting=csv.QUOTE_MINIMAL)
    print(f"Saved: {path} | rows={len(df):,} cols={len(df.columns)}")

def sample_blocks(
    df: pd.DataFrame,
    total_records: int,
    fixed_records: int,
    block_size: int,
    rng: np.random.Generator,
) -> pd.DataFrame:
    """
    Take first `fixed_records` rows (if available), then sample N random blocks of size `block_size`
    from the remaining rows without overlap. Adjusts gracefully if the dataset is smaller.
    """
    n = len(df)
    if n == 0:
        return df

    # If not enough rows for the target total, just return the entire df
    if n <= total_records:
        print(f"Sampling note: dataset has only {n:,} rows <= {total_records:,}; returning full dataset.")
        return df.copy()

    # Fixed head
    fixed_end = min(fixed_records, n)
    fixed_part = df.iloc[:fixed_end]

    remaining = df.iloc[fixed_end:].reset_index(drop=True)
    remaining_n = len(remaining)

    # How many blocks can we realistically draw?
    max_blocks = remaining_n // block_size
    want_blocks = max((total_records - fixed_end) // block_size, 0)
    draw_blocks = min(max_blocks, want_blocks)

    # Choose block starting indices without overlap (step = block_size)
    possible_starts = np.arange(0, remaining_n - block_size + 1, block_size)
    if len(possible_starts) == 0 or draw_blocks == 0:
        sampled_part = remaining
    else:
        chosen_idx = rng.choice(possible_starts, size=draw_blocks, replace=False)
        blocks = [remaining.iloc[s : s + block_size] for s in chosen_idx]
        sampled_part = pd.concat(blocks, axis=0).sort_index()

    out = pd.concat([fixed_part, sampled_part], axis=0)
    # If we overshot (unlikely), trim
    if len(out) > total_records:
        out = out.iloc[:total_records]
    return out.reset_index(drop=True)

# ────── Main pipeline ───────────────────────────────────────────────────────────
def main():
    # 1) Read games CSV
    df_games = read_games_csv(input_dir)
    if df_games is None:
        print("ERROR: Could not find a games CSV in the input directory. "
              "Expected something like 'games_cleaned.csv'.")
        return

    time_helper("Files were read.")

    # 2) Expand price_overview JSON and clean languages (if present)
    df_games = expand_price_overview(df_games)
    df_games = expand_languages(df_games)

    # 3) Save cleaned data
    cleaned_path = output_dir / "games_cleaned.csv"
    to_csv(df_games, cleaned_path)

    # 4) Optional sampling (only meaningful when df is large)
    sampled_df = sample_blocks(
        df_games,
        total_records=TOTAL_RECORDS,
        fixed_records=FIXED_RECORDS,
        block_size=BLOCK_SIZE,
        rng=RNG,
    )
    sampled_path = output_dir / "games_sampled.csv"
    to_csv(sampled_df, sampled_path)

    time_helper("Processing completed.")

if __name__ == "__main__":
    main()


Loaded: games.csv | rows=140,082 cols=7
── Files were read.   Time used: 0m  0.68s
Saved: /Users/timtoepper/Downloads/Banks_Assignment_Tim/sampled_outputs/games_cleaned.csv | rows=140,082 cols=14
Saved: /Users/timtoepper/Downloads/Banks_Assignment_Tim/sampled_outputs/games_sampled.csv | rows=10,000 cols=14
── Processing completed.   Time used: 0m  10.72s


PReprocessing merger final + Values

In [5]:
# Complete Data Cleaning and Merging Pipeline - Enhanced Version
# Handles all 8 data files with USD conversion and comprehensive cleaning

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import csv
import re
from datetime import datetime
warnings.filterwarnings('ignore')

# Configuration - MODIFIED TO WORK AFTER SCRIPT 6
DATA_DIR = Path("/Users/timtoepper/Downloads/Banks_Assignment_Tim")

# Check if Script 6 has been run
sampled_outputs_dir = DATA_DIR / "sampled_outputs"
if sampled_outputs_dir.exists() and (sampled_outputs_dir / "games_cleaned.csv").exists():
    print("Found Script 6 outputs - using cleaned games data")
    GAMES_FILE = sampled_outputs_dir / "games_cleaned.csv"
    USE_SAMPLED = False  # Set to True if you want to use the 10k sample instead
    if USE_SAMPLED:
        GAMES_FILE = sampled_outputs_dir / "games_sampled.csv"
else:
    print("Script 6 outputs not found - using original games.csv")
    GAMES_FILE = DATA_DIR / "games.csv"

print(f"Data directory: {DATA_DIR}")
print(f"Games file: {GAMES_FILE}")
print("=" * 80)

# ============================================
# CURRENCY CONVERSION SETUP
# ============================================

# Historical FX rates as of 2025-08-01 (fallback rates)
FX_RATES_TO_USD = {
    'USD': 1.0,
    'EUR': 1.09,
    'GBP': 1.28,
    'JPY': 0.0067,
    'CNY': 0.14,
    'CAD': 0.72,
    'AUD': 0.65,
    'RUB': 0.011,
    'BRL': 0.18,
    'INR': 0.012,
    'KRW': 0.00077,
    'MXN': 0.055,
    'SEK': 0.094,
    'CHF': 1.16,
    'PLN': 0.25,
    'TRY': 0.030,
    'ARS': 0.0011,
    'NZD': 0.60,
    'SGD': 0.74,
    'HKD': 0.13,
    'NOK': 0.093,
    'DKK': 0.146,
    'ZAR': 0.054,
    'THB': 0.028,
    'IDR': 0.000063,
    'MYR': 0.22,
    'PHP': 0.018,
    'CZK': 0.044,
    'HUF': 0.0028,
    'ILS': 0.27,
    'CLP': 0.0011,
    'AED': 0.27,
    'SAR': 0.27,
    'RON': 0.22,
    'COP': 0.00024,
    'VND': 0.000040,
    'UAH': 0.024,
    'EGP': 0.021,
    'PKR': 0.0036,
    'BGN': 0.56,
    'HRK': 0.14,
    'LTL': 0.32,  # Historical
    'LVL': 1.55,  # Historical
    'EEK': 0.070, # Historical
}

def normalize_currency_code(currency):
    r"""Normalize currency codes, handling \N, 0, empty values"""
    if pd.isna(currency) or currency == '' or currency == '\\N' or currency == 'N' or currency == '0' or currency == 0:
        return 'USD'  # Default to USD
    currency = str(currency).strip().upper()
    if currency in FX_RATES_TO_USD:
        return currency
    return 'USD'  # Default for unknown currencies

def convert_to_usd(amount, currency):
    """Convert any amount to USD using FX rates"""
    if pd.isna(amount) or amount == 0:
        return 0
    currency = normalize_currency_code(currency)
    rate = FX_RATES_TO_USD.get(currency, 1.0)
    return amount * rate

# ============================================
# HELPER FUNCTIONS FOR DATA CLEANING
# ============================================

def clean_numeric_column(series):
    r"""Clean numeric columns: handle \N, convert to numeric"""
    return pd.to_numeric(
        series.replace('\\N', np.nan).replace('', np.nan),
        errors='coerce'
    )

def clean_text_column(series):
    r"""Clean text columns: handle \N, strip whitespace"""
    return series.replace('\\N', '').replace('', '').str.strip()

def parse_owners_range(x):
    r"""Parse Steam owner ranges like '10,000,000 .. 20,000,000'"""
    if pd.isna(x) or x == '' or x == '\\N':
        return 0
    x = str(x).replace(',', '')
    if '..' in x:
        parts = x.split('..')
        if len(parts) == 2:
            try:
                low = float(parts[0].strip())
                high = float(parts[1].strip())
                return (low + high) / 2  # Average of range
            except:
                return 0
    try:
        return float(x)
    except:
        return 0

def clean_review_text(text):
    """Clean HTML tags and special characters from review text"""
    if pd.isna(text):
        return ''
    
    text = str(text)
    if text == '\\N' or text == 'N' or text == '':
        return ''
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)
    
    # Remove HTML entities
    text = text.replace('&quot;', '"')
    text = text.replace('&amp;', '&')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    text = text.replace('&nbsp;', ' ')
    
    # Remove escape characters
    text = text.replace('\\n', ' ')
    text = text.replace('\\r', ' ')
    text = text.replace('\\t', ' ')
    text = text.replace('\\"', '"')
    text = text.replace('\\', '')
    
    # Remove <br /> and <br/> specifically
    text = re.sub(r'<br\s*/?>', ' ', text)
    
    # Clean up multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def clean_description_text(text):
    """Clean description text similar to review text cleaning"""
    if pd.isna(text):
        return ''
    
    text = str(text)
    if text == '\\N' or text == 'N' or text == '':
        return ''
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)
    
    # Remove HTML entities
    text = text.replace('&quot;', '"')
    text = text.replace('&amp;', '&')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    text = text.replace('&nbsp;', ' ')
    text = text.replace('&#39;', "'")
    
    # Remove escape characters
    text = text.replace('\\n', ' ')
    text = text.replace('\\r', ' ')
    text = text.replace('\\t', ' ')
    text = text.replace('\\"', '"')
    text = text.replace('\\', '')
    
    # Remove <br /> and <br/> specifically
    text = re.sub(r'<br\s*/?>', ' ', text)
    
    # Clean up multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()




# ============================================
# 1. LOAD AND CLEAN GAMES.CSV
# ============================================
print("\n1. LOADING AND CLEANING GAMES.CSV")
print("-" * 60)

games_df = pd.read_csv(GAMES_FILE, low_memory=False)
print(f"Loaded: {len(games_df):,} games from {GAMES_FILE.name}")

# Note: If coming from Script 6, some cleaning may already be done
if "price_final" in games_df.columns and "price_final_cents" not in games_df.columns:
    print("  Note: Prices already expanded from JSON (Script 6 output)")

# Clean app_id
games_df['app_id'] = clean_numeric_column(games_df['app_id'])
games_df = games_df.dropna(subset=['app_id'])
games_df['app_id'] = games_df['app_id'].astype(int)

# Clean and normalize currency
if 'currency' in games_df.columns:
    games_df['currency'] = games_df['currency'].apply(normalize_currency_code)
else:
    games_df['currency'] = 'USD'

# Clean prices
for col in ['price_final_cents', 'price_initial_cents', 'discount_percent']:
    if col in games_df.columns:
        games_df[col] = clean_numeric_column(games_df[col])

# Convert cents to dollars in original currency
if 'price_final_cents' in games_df.columns:
    games_df['price_final'] = games_df['price_final_cents'] / 100
if 'price_initial_cents' in games_df.columns:
    games_df['price_initial'] = games_df['price_initial_cents'] / 100

# Convert to USD
games_df['price_final_usd'] = games_df.apply(
    lambda row: convert_to_usd(row.get('price_final', 0), row.get('currency', 'USD')), axis=1
)
games_df['price_initial_usd'] = games_df.apply(
    lambda row: convert_to_usd(row.get('price_initial', 0), row.get('currency', 'USD')), axis=1
)

# Calculate average price for revenue calculations
games_df['price_avg_usd'] = (games_df['price_final_usd'] + games_df['price_initial_usd']) / 2

# Parse release date
if 'release_date' in games_df.columns:
    games_df['release_date'] = pd.to_datetime(games_df['release_date'], errors='coerce')

# Clean is_free flag
if 'is_free' in games_df.columns:
    games_df['is_free'] = clean_numeric_column(games_df['is_free']).fillna(0).astype(bool)

print(f"Cleaned games: {len(games_df):,} rows × {len(games_df.columns)} columns")

# Initialize merged dataframe with ALL games columns
merged = games_df.copy()

# [REST OF THE SCRIPT CONTINUES EXACTLY AS IN THE DOCUMENT FROM LINE 251 ONWARDS]

# ============================================
# 2. LOAD AND CLEAN STEAMSPY_INSIGHTS.CSV
# ============================================
print("\n2. LOADING AND CLEANING STEAMSPY_INSIGHTS.CSV")
print("-" * 60)

try:
    # Use csv.DictReader for malformed CSV
    steamspy_data = []
    with open(DATA_DIR / "steamspy_insights.csv", 'r', encoding='utf-8', errors='ignore') as f:
        reader = csv.DictReader(f)
        for i, row in enumerate(reader):
            if i % 25000 == 0 and i > 0:
                print(f"  Processing row {i:,}...")
            # Clean each row
            clean_row = {}
            for key, value in row.items():
                if value == '\\N' or value == 'N' or value == '':
                    clean_row[key] = None
                else:
                    clean_row[key] = value
            steamspy_data.append(clean_row)
    
    steamspy_df = pd.DataFrame(steamspy_data)
    print(f"Loaded: {len(steamspy_df):,} records")
    
    # Clean app_id
    steamspy_df['app_id'] = clean_numeric_column(steamspy_df['app_id'])
    steamspy_df = steamspy_df.dropna(subset=['app_id'])
    steamspy_df['app_id'] = steamspy_df['app_id'].astype(int)
    
    # Parse owners range (average of range)
    if 'owners_range' in steamspy_df.columns:
        steamspy_df['owners_avg'] = steamspy_df['owners_range'].apply(parse_owners_range)
        print(f"  Parsed owners - Average: {steamspy_df['owners_avg'].mean():,.0f}")
    
    # Clean numeric columns
    numeric_cols = ['concurrent_users_yesterday', 'playtime_average_forever', 
                   'playtime_average_2weeks', 'playtime_median_forever', 
                   'playtime_median_2weeks', 'price', 'initial_price', 'discount']
    
    for col in numeric_cols:
        if col in steamspy_df.columns:
            steamspy_df[col] = clean_numeric_column(steamspy_df[col])
    
    # Convert steamspy prices to USD
    # Assuming steamspy prices are in cents
    if 'price' in steamspy_df.columns:
        if steamspy_df['price'].max() > 1000:  # Likely in cents
            steamspy_df['price'] = steamspy_df['price'] / 100
        steamspy_df['price_usd'] = steamspy_df['price']  # Assume USD for steamspy
    
    if 'initial_price' in steamspy_df.columns:
        if steamspy_df['initial_price'].max() > 1000:
            steamspy_df['initial_price'] = steamspy_df['initial_price'] / 100
        steamspy_df['initial_price_usd'] = steamspy_df['initial_price']  # Assume USD
    
    # Clean text columns
    for col in ['developer', 'publisher', 'languages', 'genres']:
        if col in steamspy_df.columns:
            steamspy_df[col] = clean_text_column(steamspy_df[col])
    
    # Merge with main dataset - keep ALL columns
    steamspy_cols_to_merge = steamspy_df.columns.tolist()
    merged = pd.merge(merged, steamspy_df[steamspy_cols_to_merge], 
                      on='app_id', how='left', suffixes=('', '_steamspy'))
    
    print(f"Merged steamspy: {merged.shape[0]:,} rows × {merged.shape[1]} columns")
    
except Exception as e:
    print(f"Error with steamspy: {e}")

# ============================================
# 3. LOAD AND CLEAN REVIEWS.CSV
# ============================================
print("\n3. LOADING AND CLEANING REVIEWS.CSV")
print("-" * 60)

try:
    reviews_df = pd.read_csv(DATA_DIR / "reviews.csv", 
                            on_bad_lines='skip',
                            engine='python',
                            na_values=['\\N', '\\\\N', 'N', 'NULL', ''],
                            keep_default_na=True)
    print(f"Loaded: {len(reviews_df):,} records")
    
    # Clean app_id
    reviews_df['app_id'] = clean_numeric_column(reviews_df['app_id'])
    reviews_df = reviews_df.dropna(subset=['app_id'])
    reviews_df['app_id'] = reviews_df['app_id'].astype(int)
    
    # Clean all numeric columns
    numeric_cols = ['review_score', 'positive', 'negative', 'total', 
                   'metacritic_score', 'recommendations', 'steamspy_user_score', 
                   'steamspy_score_rank', 'steamspy_positive', 'steamspy_negative']
    
    for col in numeric_cols:
        if col in reviews_df.columns:
            reviews_df[col] = clean_numeric_column(reviews_df[col])
    
    # Clean review text
    if 'reviews' in reviews_df.columns:
        print("  Cleaning reviews text...")
        reviews_df['reviews'] = reviews_df['reviews'].fillna('')
        reviews_df['reviews'] = reviews_df['reviews'].replace('\\N', '')
        reviews_df['reviews_clean'] = reviews_df['reviews'].apply(clean_review_text)
        valid_reviews = (reviews_df['reviews_clean'] != '').sum()
        print(f"  Cleaned reviews text: {valid_reviews:,} valid reviews")
    
    if 'review_score_description' in reviews_df.columns:
        reviews_df['review_score_description'] = clean_text_column(reviews_df['review_score_description'])
    
    # Calculate metrics
    if 'positive' in reviews_df.columns and 'negative' in reviews_df.columns:
        reviews_df['total_reviews'] = reviews_df['positive'].fillna(0) + reviews_df['negative'].fillna(0)
        reviews_df['positive_ratio'] = reviews_df['positive'] / (reviews_df['total_reviews'] + 1)
        reviews_df['positive_ratio'] = reviews_df['positive_ratio'].fillna(0)
    
    # Merge with main dataset - keep ALL columns
    reviews_cols_to_merge = reviews_df.columns.tolist()
    merged = pd.merge(merged, reviews_df[reviews_cols_to_merge], 
                      on='app_id', how='left', suffixes=('', '_reviews'))
    
    print(f"Merged reviews: {merged.shape[0]:,} rows × {merged.shape[1]} columns")
    
except Exception as e:
    print(f"Error with reviews: {e}")

# ============================================
# 4. LOAD AND CLEAN CATEGORIES.CSV
# ============================================
print("\n4. LOADING AND CLEANING CATEGORIES.CSV")
print("-" * 60)

try:
    cat_df = pd.read_csv(DATA_DIR / "categories.csv", on_bad_lines='skip', engine='python')
    print(f"Loaded: {len(cat_df):,} records")
    
    # Clean app_id
    cat_df['app_id'] = clean_numeric_column(cat_df['app_id'])
    cat_df = cat_df.dropna(subset=['app_id'])
    cat_df['app_id'] = cat_df['app_id'].astype(int)
    
    # Clean category names
    if 'category' in cat_df.columns:
        cat_df['category'] = clean_text_column(cat_df['category'])
        cat_df = cat_df[cat_df['category'] != '']
    
    # Aggregate categories - keep ALL, dedupe and sort
    cat_agg = cat_df.groupby('app_id')['category'].apply(
        lambda x: '|'.join(sorted(set(x.dropna())))
    ).reset_index()
    cat_agg.columns = ['app_id', 'categories_all']
    
    # Extract features
    cat_agg['category_count'] = cat_agg['categories_all'].apply(
        lambda x: len(x.split('|')) if x else 0
    )
    cat_agg['has_multiplayer'] = cat_agg['categories_all'].str.contains('Multi-player|Multiplayer', case=False, na=False)
    cat_agg['has_singleplayer'] = cat_agg['categories_all'].str.contains('Single-player|Singleplayer', case=False, na=False)
    cat_agg['has_coop'] = cat_agg['categories_all'].str.contains('Co-op', case=False, na=False)
    cat_agg['has_vr'] = cat_agg['categories_all'].str.contains('VR', case=False, na=False)
    cat_agg['has_controller'] = cat_agg['categories_all'].str.contains('controller', case=False, na=False)
    
    # Merge
    merged = pd.merge(merged, cat_agg, on='app_id', how='left')
    print(f"Merged categories: {merged.shape[0]:,} rows × {merged.shape[1]} columns")
    
except Exception as e:
    print(f"Error with categories: {e}")

# ============================================
# 5. LOAD AND CLEAN TAGS.CSV - ALL TAGS
# ============================================
print("\n5. LOADING AND CLEANING TAGS.CSV (ALL TAGS)")
print("-" * 60)

try:
    tags_df = pd.read_csv(DATA_DIR / "tags.csv", on_bad_lines='skip', engine='python')
    print(f"Loaded: {len(tags_df):,} records")
    
    # Clean app_id
    tags_df['app_id'] = clean_numeric_column(tags_df['app_id'])
    tags_df = tags_df.dropna(subset=['app_id'])
    tags_df['app_id'] = tags_df['app_id'].astype(int)
    
    # Clean tag names
    if 'tag' in tags_df.columns:
        tags_df['tag'] = clean_text_column(tags_df['tag'])
        tags_df = tags_df[tags_df['tag'] != '']
    
    # Get ALL tags per game, deduped and sorted alphabetically
    tags_agg = tags_df.groupby('app_id')['tag'].apply(
        lambda x: '|'.join(sorted(set(x.dropna())))
    ).reset_index()
    tags_agg.columns = ['app_id', 'tags_all']
    
    # Count unique tags
    tags_agg['tag_count'] = tags_agg['tags_all'].apply(
        lambda x: len(x.split('|')) if x else 0
    )
    
    # Extract genre indicators from ALL tags
    tags_agg['is_indie'] = tags_agg['tags_all'].str.contains('Indie', case=False, na=False)
    tags_agg['is_action'] = tags_agg['tags_all'].str.contains('Action', case=False, na=False)
    tags_agg['is_adventure'] = tags_agg['tags_all'].str.contains('Adventure', case=False, na=False)
    tags_agg['is_rpg'] = tags_agg['tags_all'].str.contains('RPG', case=False, na=False)
    tags_agg['is_strategy'] = tags_agg['tags_all'].str.contains('Strategy', case=False, na=False)
    tags_agg['is_simulation'] = tags_agg['tags_all'].str.contains('Simulation', case=False, na=False)
    tags_agg['is_puzzle'] = tags_agg['tags_all'].str.contains('Puzzle', case=False, na=False)
    
    # Merge
    merged = pd.merge(merged, tags_agg, on='app_id', how='left')
    print(f"Merged tags: {merged.shape[0]:,} rows × {merged.shape[1]} columns")
    print(f"  Average tags per game: {tags_agg['tag_count'].mean():.1f}")
    print(f"  Max tags on a game: {tags_agg['tag_count'].max()}")
    
except Exception as e:
    print(f"Error with tags: {e}")

# ============================================
# 6. LOAD AND CLEAN GENRES.CSV
# ============================================
print("\n6. LOADING AND CLEANING GENRES.CSV")
print("-" * 60)

try:
    genres_df = pd.read_csv(DATA_DIR / "genres.csv", on_bad_lines='skip', engine='python')
    print(f"Loaded: {len(genres_df):,} records")
    
    # Clean app_id
    genres_df['app_id'] = clean_numeric_column(genres_df['app_id'])
    genres_df = genres_df.dropna(subset=['app_id'])
    genres_df['app_id'] = genres_df['app_id'].astype(int)
    
    # Clean genre names
    if 'genre' in genres_df.columns:
        genres_df['genre'] = clean_text_column(genres_df['genre'])
        genres_df = genres_df[genres_df['genre'] != '']
    
    # Aggregate genres
    genres_agg = genres_df.groupby('app_id')['genre'].apply(
        lambda x: '|'.join(sorted(set(x.dropna())))
    ).reset_index()
    genres_agg.columns = ['app_id', 'genres_all']
    
    # Get primary genre
    genres_agg['primary_genre'] = genres_agg['genres_all'].str.split('|').str[0]
    genres_agg['genre_count'] = genres_agg['genres_all'].apply(
        lambda x: len(x.split('|')) if x else 0
    )
    
    # Merge
    merged = pd.merge(merged, genres_agg, on='app_id', how='left')
    print(f"Merged genres: {merged.shape[0]:,} rows × {merged.shape[1]} columns")
    
except Exception as e:
    print(f"Error with genres: {e}")

# ============================================
# 7. LOAD AND CLEAN DESCRIPTIONS.CSV (FULL)
# ============================================
print("\n7. LOADING AND CLEANING DESCRIPTIONS.CSV (FULL DATASET)")
print("-" * 60)

try:
    desc_df = pd.read_csv(DATA_DIR / "descriptions.csv", 
                         on_bad_lines='skip', 
                         engine='python',
                         na_values=['\\N', 'NULL'])
    print(f"Loaded: {len(desc_df):,} records")
    
    # Clean app_id
    desc_df['app_id'] = clean_numeric_column(desc_df['app_id'])
    desc_df = desc_df.dropna(subset=['app_id'])
    desc_df['app_id'] = desc_df['app_id'].astype(int)
    
    # Clean description fields and create clean versions
    description_fields = ['summary', 'extensive', 'about', 'short_description', 'detailed_description', 'about_the_game']
    
    for col in description_fields:
        if col in desc_df.columns:
            # Create cleaned version
            desc_df[f'{col}_clean'] = desc_df[col].apply(clean_description_text)
            # Add length of cleaned version
            desc_df[f'{col}_length'] = desc_df[f'{col}_clean'].str.len().fillna(0)
            # Drop the original dirty column
            desc_df = desc_df.drop(columns=[col])
            print(f"  Cleaned {col}: {(desc_df[f'{col}_clean'] != '').sum():,} valid descriptions")
    
    # Keep ALL description columns
    desc_cols_to_merge = desc_df.columns.tolist()
    merged = pd.merge(merged, desc_df[desc_cols_to_merge], 
                      on='app_id', how='left', suffixes=('', '_desc'))
    
    print(f"Merged descriptions: {merged.shape[0]:,} rows × {merged.shape[1]} columns")
    
except Exception as e:
    print(f"Error with descriptions: {e}")

# ============================================
# 8. LOAD AND CLEAN PROMOTIONAL.CSV (FULL)
# ============================================
print("\n8. LOADING AND CLEANING PROMOTIONAL.CSV (FULL DATASET)")
print("-" * 60)

try:
    promo_df = pd.read_csv(DATA_DIR / "promotional.csv", 
                          on_bad_lines='skip', 
                          engine='python',
                          na_values=['\\N', 'NULL'])
    print(f"Loaded: {len(promo_df):,} records")
    
    # Clean app_id
    promo_df['app_id'] = clean_numeric_column(promo_df['app_id'])
    promo_df = promo_df.dropna(subset=['app_id'])
    promo_df['app_id'] = promo_df['app_id'].astype(int)
    
    # Aggregate promotional materials per game
    promo_count = promo_df.groupby('app_id').agg({
        'app_id': 'count'  # Count promotional items
    }).rename(columns={'app_id': 'promo_material_count'})
    
    # Get promotional content types if available
    if 'type' in promo_df.columns:
        promo_types = promo_df.groupby('app_id')['type'].apply(
            lambda x: '|'.join(sorted(set(x.dropna().astype(str))))
        ).reset_index()
        promo_types.columns = ['app_id', 'promo_types']
        promo_count = pd.merge(promo_count, promo_types, on='app_id', how='left')
    
    # Keep all promotional columns if they have other data
    promo_cols_to_merge = promo_count.columns.tolist()
    merged = pd.merge(merged, promo_count, on='app_id', how='left')
    
    print(f"Merged promotional: {merged.shape[0]:,} rows × {merged.shape[1]} columns")
    
except Exception as e:
    print(f"Error with promotional: {e}")

# ============================================
# 9. CALCULATE KPIs WITH USD
# ============================================
print("\n9. CALCULATING KEY PERFORMANCE INDICATORS (USD-BASED)")
print("-" * 60)

# Estimated revenue in USD
if 'owners_avg' in merged.columns:
    # Use average of initial and final USD prices
    merged['estimated_revenue_usd'] = merged['owners_avg'] * merged['price_avg_usd']
    merged['estimated_revenue_usd'] = merged['estimated_revenue_usd'].fillna(0)
    total_revenue_usd = merged['estimated_revenue_usd'].sum()
    print(f"Estimated total revenue (USD): ${total_revenue_usd:,.0f}")

# Game age
if 'release_date' in merged.columns:
    merged['game_age_years'] = (pd.Timestamp.now() - merged['release_date']).dt.days / 365.25
    merged['game_age_years'] = merged['game_age_years'].clip(lower=0).fillna(0)
    print(f"Average game age: {merged['game_age_years'].mean():.1f} years")

# Success metrics
if 'total_reviews' in merged.columns:
    threshold = merged['total_reviews'].quantile(0.75)
    merged['is_successful'] = merged['total_reviews'] > threshold
    print(f"Successful games (>75th percentile): {merged['is_successful'].sum():,}")

if 'positive_ratio' in merged.columns:
    merged['quality_score'] = merged['positive_ratio'] * 100
    print(f"Average quality score: {merged['quality_score'].mean():.1f}/100")

# Hit games (top 10% by ownership)
if 'owners_avg' in merged.columns:
    hit_threshold = merged['owners_avg'].quantile(0.90)
    merged['is_hit'] = merged['owners_avg'] > hit_threshold
    print(f"Hit games (>90th percentile): {merged['is_hit'].sum():,}")

# ============================================
# 9A. CLEAN PUBLISHER DATA
# ============================================
if 'publisher' in merged.columns:
    # Clean publisher names
    merged['publisher'] = merged['publisher'].fillna('Unknown')
    merged['publisher'] = merged['publisher'].str.strip()
    
    # Filter out generic/bad publisher names
    bad_publishers = ['Unknown', '', 'N/A', '\\N', 'None']
    valid_publishers = ~merged['publisher'].isin(bad_publishers)
    print(f"Valid publishers: {valid_publishers.sum():,} games")

# ============================================
# 9B. CALCULATE ADVANCED M&A KPIs
# ============================================
print("\nCALCULATING ADVANCED M&A METRICS")
print("-" * 60)

# 1. Content Production Metrics - VECTORIZED
print("Calculating Content Production Metrics...")
if 'release_date' in merged.columns and 'publisher' in merged.columns:
    # Vectorized calculations - much faster!
    merged['games_last_3y'] = merged.groupby('publisher')['game_age_years'].transform(
        lambda x: (x <= 3).sum()
    )
    merged['release_cadence'] = merged['games_last_3y'] / 3
    merged['recent_games_ratio'] = merged.groupby('publisher')['game_age_years'].transform(
        lambda x: (x <= 2).sum() / len(x) if len(x) > 0 else 0
    )

# 2. Subscriber Retention Power
print("Calculating Retention Metrics...")
if all(col in merged.columns for col in ['playtime_median_forever', 'playtime_average_forever', 
                                          'concurrent_users_yesterday', 'owners_avg']):
    merged['retention_score'] = (
        merged['playtime_median_forever'].fillna(0) * 0.3 +
        merged['playtime_average_forever'].fillna(0) * 0.3 +
        (merged['concurrent_users_yesterday'] / (merged['owners_avg'] + 1)) * 1000 * 0.4
    )
    
    # Replay value relative to genre
    if 'primary_genre' in merged.columns:
        genre_avg_playtime = merged.groupby('primary_genre')['playtime_average_forever'].transform('median')
        merged['replay_value'] = merged['playtime_average_forever'] / (genre_avg_playtime + 1)

# 3. Franchise Detection
print("Detecting Franchises...")
if 'name' in merged.columns:
    # Simple franchise detection based on common patterns
    merged['is_sequel'] = merged['name'].str.contains(
        r'\b[2-9]|\bII|\bIII|\bIV|\bV\b|:|\s-\s', 
        regex=True, na=False
    )
    
    # Calculate sequel performance if quality scores exist
    if 'quality_score' in merged.columns:
        sequel_quality = merged[merged['is_sequel']]['quality_score'].mean()
        original_quality = merged[~merged['is_sequel']]['quality_score'].mean()
        merged['sequel_performance_ratio'] = sequel_quality / (original_quality + 1)

# 4. Audience Diversity Metrics
print("Calculating Audience Metrics...")
if 'languages_clean' in merged.columns:
    # Count languages (already have this, but calculate if missing)
    if 'language_count' not in merged.columns:
        merged['language_count'] = merged['languages_clean'].apply(
            lambda x: len(x) if isinstance(x, list) else 0
        )
    
    # Language market score (languages × reach)
    merged['language_market_score'] = merged['language_count'] * merged['owners_avg']

# Content maturity diversity
if 'tags_all' in merged.columns:
    merged['has_mature_content'] = merged['tags_all'].str.contains(
        'Mature|Adult|Gore|Violence|Sexual|Nudity', case=False, na=False
    )
    merged['has_family_content'] = merged['tags_all'].str.contains(
        'Family|Kids|Casual|Educational|Cute', case=False, na=False
    )

# 5. Financial Efficiency Metrics
print("Calculating Financial Efficiency...")
if all(col in merged.columns for col in ['category_count', 'language_count', 
                                          'has_multiplayer', 'tag_count']):
    # FIX: Fill NaN values BEFORE converting to int
    merged['has_multiplayer'] = merged['has_multiplayer'].fillna(False)
    
    # Development complexity proxy
    merged['development_complexity'] = (
        merged['category_count'].fillna(0) * 10 +
        merged['language_count'].fillna(0) * 5 +
        merged['has_multiplayer'].astype(int) * 50 +
        merged['tag_count'].fillna(0) * 2
    )
    
    merged['roi_efficiency'] = merged['estimated_revenue_usd'] / (merged['development_complexity'] + 1)
    
    # CAC efficiency
    if 'promo_material_count' in merged.columns:
        merged['cac_efficiency'] = merged['owners_avg'] / (merged['promo_material_count'].fillna(1) + 1)

# 6. Risk Assessment Metrics
print("Calculating Risk Metrics...")

# Platform dependency risk
if 'tags_all' in merged.columns:
    merged['is_pc_exclusive'] = ~merged['tags_all'].str.contains(
        'Console|PlayStation|Xbox|Switch|Mobile', case=False, na=False
    )
    
    # Technology risk
    merged['uses_old_tech'] = merged['tags_all'].str.contains(
        'Retro|Classic|Old School|8-bit|16-bit|Pixel', case=False, na=False
    )
    merged['uses_new_tech'] = merged['tags_all'].str.contains(
        'VR|Ray Tracing|DLSS|Next-Gen|RTX|4K', case=False, na=False
    )

# Controversy risk from reviews
if 'reviews_clean' in merged.columns:
    merged['has_controversy'] = merged['reviews_clean'].str.contains(
        'controversy|lawsuit|stolen|copyright|plagiar|scam|fraud|broken',
        case=False, na=False
    )

# 7. Advanced Success Categories
print("Calculating Advanced Success Metrics...")

# Commercial Success/Failure
if 'estimated_revenue_usd' in merged.columns:
    merged['is_commercial_success'] = merged['estimated_revenue_usd'] > 100000
    merged['is_commercial_flop'] = (
        (merged['estimated_revenue_usd'] < 5000) & 
        (merged['game_age_years'] > 0.25) &
        (~merged['is_free'])
    )

# Critical Success/Failure
if 'positive_ratio' in merged.columns and 'total_reviews' in merged.columns:
    merged['is_critical_success'] = (
        (merged['positive_ratio'] > 0.8) & 
        (merged['total_reviews'] >= 50)
    )
    merged['is_critical_flop'] = (
        (merged['positive_ratio'] < 0.4) & 
        (merged['total_reviews'] >= 20)
    )

# Combined categories
merged['is_commercial_success'] = merged['is_commercial_success'].fillna(False)
merged['is_critical_success'] = merged['is_critical_success'].fillna(False)
merged['is_commercial_flop'] = merged['is_commercial_flop'].fillna(False)
merged['is_critical_flop'] = merged['is_critical_flop'].fillna(False)

merged['is_total_success'] = merged['is_commercial_success'] & merged['is_critical_success']
merged['is_total_flop'] = merged['is_commercial_flop'] & merged['is_critical_flop']
merged['is_hidden_gem'] = merged['is_critical_success'] & merged['is_commercial_flop']

# Dead game detection
if 'concurrent_users_yesterday' in merged.columns:
    merged['is_currently_dead'] = (
        (merged['concurrent_users_yesterday'] == 0) & 
        (merged['game_age_years'] > 0.5)
    )
    merged['is_zombie_game'] = (
        (merged['concurrent_users_yesterday'] == 0) &
        (merged['owners_avg'] < 1000) &
        (merged['total_reviews'] < 10) &
        (merged['game_age_years'] > 0.5)
    )

print(f"Commercial successes: {merged['is_commercial_success'].sum():,}")
print(f"Critical successes: {merged['is_critical_success'].sum():,}")
print(f"Hidden gems: {merged['is_hidden_gem'].sum():,}")
print(f"Dead games: {merged['is_currently_dead'].sum() if 'is_currently_dead' in merged.columns else 0:,}")

# ============================================
# 10. ENHANCED PUBLISHER AGGREGATION & ANALYSIS
# ============================================
print("\nCALCULATING PUBLISHER METRICS FOR M&A")
print("-" * 60)

# Define output directory
output_dir = DATA_DIR / "output"
output_dir.mkdir(exist_ok=True)

if 'publisher' in merged.columns and valid_publishers.sum() > 0:
    # Basic publisher metrics
    publisher_metrics = merged[valid_publishers].groupby('publisher').agg({
        'app_id': 'count',
        'owners_avg': lambda x: x.fillna(0).sum(),
        'estimated_revenue_usd': lambda x: x.fillna(0).sum(),
        'positive_ratio': lambda x: x[x > 0].mean() if len(x[x > 0]) > 0 else 0,
        'total_reviews': lambda x: x.fillna(0).sum(),
        'has_multiplayer': lambda x: x.sum() if 'has_multiplayer' in merged.columns else 0,
        'has_vr': lambda x: x.sum() if 'has_vr' in merged.columns else 0,
        'is_successful': lambda x: x.sum() if 'is_successful' in merged.columns else 0,
        'tag_count': lambda x: x.mean() if 'tag_count' in merged.columns else 0,
    }).round(2)
    
    publisher_metrics.columns = ['game_count', 'total_players', 'total_revenue_usd', 
                                 'avg_rating', 'total_reviews', 'multiplayer_games', 
                                 'vr_games', 'successful_games', 'avg_tags_per_game']
    
    # Add calculated metrics
    publisher_metrics['success_rate'] = (publisher_metrics['successful_games'] / publisher_metrics['game_count'] * 100).round(1)
    publisher_metrics['avg_revenue_per_game_usd'] = (publisher_metrics['total_revenue_usd'] / publisher_metrics['game_count']).round(0)
    
    # Sort by USD revenue
    publisher_metrics = publisher_metrics.sort_values('total_revenue_usd', ascending=False)
    
    # Save basic publisher metrics
    publisher_file = output_dir / "publisher_metrics_cleaned.csv"
    publisher_metrics.to_csv(publisher_file)
    print(f"Saved publisher metrics: {publisher_file}")
    
    # Enhanced M&A-specific aggregation
    publisher_ma_metrics = merged[valid_publishers].groupby('publisher').agg({
        'app_id': 'count',
        'owners_avg': lambda x: x.fillna(0).sum(),
        'estimated_revenue_usd': lambda x: x.fillna(0).sum(),
        'release_cadence': 'first' if 'release_cadence' in merged.columns else lambda x: 0,
        'recent_games_ratio': 'first' if 'recent_games_ratio' in merged.columns else lambda x: 0,
        'retention_score': 'mean' if 'retention_score' in merged.columns else lambda x: 0,
        'replay_value': 'mean' if 'replay_value' in merged.columns else lambda x: 0,
        'concurrent_users_yesterday': 'sum',
        'is_commercial_success': 'sum' if 'is_commercial_success' in merged.columns else lambda x: 0,
        'is_critical_success': 'sum' if 'is_critical_success' in merged.columns else lambda x: 0,
        'is_hidden_gem': 'sum' if 'is_hidden_gem' in merged.columns else lambda x: 0,
        'is_total_flop': 'sum' if 'is_total_flop' in merged.columns else lambda x: 0,
        'is_zombie_game': 'sum' if 'is_zombie_game' in merged.columns else lambda x: 0,
        'roi_efficiency': 'mean' if 'roi_efficiency' in merged.columns else lambda x: 0,
        'cac_efficiency': 'mean' if 'cac_efficiency' in merged.columns else lambda x: 0,
        'is_sequel': 'sum' if 'is_sequel' in merged.columns else lambda x: 0,
        'is_pc_exclusive': 'mean' if 'is_pc_exclusive' in merged.columns else lambda x: 0,
        'uses_new_tech': 'mean' if 'uses_new_tech' in merged.columns else lambda x: 0,
        'uses_old_tech': 'mean' if 'uses_old_tech' in merged.columns else lambda x: 0,
        'has_controversy': 'sum' if 'has_controversy' in merged.columns else lambda x: 0,
        'has_mature_content': 'mean' if 'has_mature_content' in merged.columns else lambda x: 0,
        'has_family_content': 'mean' if 'has_family_content' in merged.columns else lambda x: 0,
        'language_market_score': 'sum' if 'language_market_score' in merged.columns else lambda x: 0,
    }).round(3)
    
    # Rename columns for clarity
    publisher_ma_metrics.rename(columns={
        'app_id': 'portfolio_size',
        'owners_avg': 'total_user_base',
        'estimated_revenue_usd': 'total_revenue_usd',
        'is_commercial_success': 'commercial_success_count',
        'is_critical_success': 'critical_success_count',
        'is_hidden_gem': 'hidden_gem_count',
        'is_total_flop': 'flop_count',
        'is_zombie_game': 'zombie_count',
        'is_sequel': 'sequel_count',
        'is_pc_exclusive': 'pc_exclusive_ratio',
        'uses_new_tech': 'new_tech_ratio',
        'uses_old_tech': 'old_tech_ratio',
        'has_controversy': 'controversy_count',
        'has_mature_content': 'mature_content_ratio',
        'has_family_content': 'family_content_ratio'
    }, inplace=True)
    
    # Calculate derived metrics
    publisher_ma_metrics['hit_rate'] = (
        publisher_ma_metrics['critical_success_count'] / 
        (publisher_ma_metrics['portfolio_size'] + 0.001)
    )
    publisher_ma_metrics['commercial_success_rate'] = (
        publisher_ma_metrics['commercial_success_count'] / 
        (publisher_ma_metrics['portfolio_size'] + 0.001)
    )
    publisher_ma_metrics['zombie_rate'] = (
        publisher_ma_metrics['zombie_count'] / 
        (publisher_ma_metrics['portfolio_size'] + 0.001)
    )
    publisher_ma_metrics['franchise_ratio'] = (
        publisher_ma_metrics['sequel_count'] / 
        (publisher_ma_metrics['portfolio_size'] + 0.001)
    )
    
    # Revenue concentration (Herfindahl Index)
    revenue_concentration = merged[valid_publishers].groupby('publisher').apply(
        lambda x: ((x['estimated_revenue_usd'] / (x['estimated_revenue_usd'].sum() + 0.001)) ** 2).sum()
    )
    publisher_ma_metrics['revenue_concentration'] = revenue_concentration
    
    # Calculate M&A scores with safe division
    max_portfolio = max(publisher_ma_metrics['portfolio_size'].max(), 1)
    max_roi = max(publisher_ma_metrics['roi_efficiency'].max(), 0.001)
    max_retention = max(publisher_ma_metrics['retention_score'].max(), 0.001)
    max_language = max(publisher_ma_metrics['language_market_score'].max(), 1)
    
    publisher_ma_metrics['content_score'] = (
        publisher_ma_metrics['release_cadence'].fillna(0) * 0.4 +
        publisher_ma_metrics['recent_games_ratio'].fillna(0) * 0.3 +
        (publisher_ma_metrics['portfolio_size'] / max_portfolio) * 0.3
    ) * 100
    
    publisher_ma_metrics['quality_score'] = (
        publisher_ma_metrics['hit_rate'].fillna(0) * 0.5 +
        publisher_ma_metrics['commercial_success_rate'].fillna(0) * 0.3 +
        (1 - publisher_ma_metrics['zombie_rate'].fillna(0)) * 0.2
    ) * 100
    
    # Safe revenue per user calculation
    revenue_per_user = publisher_ma_metrics['total_revenue_usd'] / (publisher_ma_metrics['total_user_base'] + 1)
    max_revenue_per_user = max(revenue_per_user.max(), 1)
    
    publisher_ma_metrics['efficiency_score'] = (
        (publisher_ma_metrics['roi_efficiency'] / max_roi) * 0.5 +
        (revenue_per_user / max_revenue_per_user) * 0.5
    ) * 100
    
    publisher_ma_metrics['strategic_fit_score'] = (
        publisher_ma_metrics['retention_score'].fillna(0) / max_retention * 0.4 +
        publisher_ma_metrics['franchise_ratio'].fillna(0) * 0.3 +
        publisher_ma_metrics['language_market_score'].fillna(0) / max_language * 0.3
    ) * 100
    
    # Risk adjustment
    publisher_ma_metrics['risk_score'] = (
        publisher_ma_metrics['revenue_concentration'].fillna(0) * 0.3 +
        publisher_ma_metrics['pc_exclusive_ratio'].fillna(0) * 0.2 +
        (publisher_ma_metrics['controversy_count'] / (publisher_ma_metrics['portfolio_size'] + 0.001)).fillna(0) * 0.3 +
        publisher_ma_metrics['zombie_rate'].fillna(0) * 0.2
    )
    
    # Final M&A score
    publisher_ma_metrics['ma_score'] = (
        publisher_ma_metrics['content_score'] * 0.25 +
        publisher_ma_metrics['quality_score'] * 0.25 +
        publisher_ma_metrics['efficiency_score'] * 0.20 +
        publisher_ma_metrics['strategic_fit_score'] * 0.20 +
        (1 - publisher_ma_metrics['risk_score']) * 100 * 0.10
    )
    
    # Assign tiers
    publisher_ma_metrics['acquisition_tier'] = pd.cut(
        publisher_ma_metrics['ma_score'],
        bins=[0, 30, 50, 70, 100],
        labels=['Avoid', 'Monitor', 'Strategic', 'Must Have']
    )
    
    # Sort by M&A score
    publisher_ma_metrics = publisher_ma_metrics.sort_values('ma_score', ascending=False)
    
    # Save enhanced publisher metrics
    ma_metrics_file = output_dir / "publisher_ma_analysis.csv"
    publisher_ma_metrics.to_csv(ma_metrics_file)
    print(f"Saved M&A analysis: {ma_metrics_file}")
    
    # Display top results
    print("\nTOP 20 PUBLISHERS BY REVENUE (USD):")
    print("=" * 80)
    for idx, row in publisher_metrics.head(20).iterrows():
        print(f"\n{str(idx)[:50]:<50}")
        print(f"  Games: {row['game_count']:.0f} | Revenue (USD): ${row['total_revenue_usd']:,.0f}")
        print(f"  Success Rate: {row['success_rate']:.1f}% | Avg Rating: {row['avg_rating']*100:.1f}%")
        print(f"  Avg Revenue/Game: ${row['avg_revenue_per_game_usd']:,.0f}")
    
    print("\nTOP 15 M&A TARGETS:")
    print("=" * 80)
    for idx, row in publisher_ma_metrics.head(15).iterrows():
        print(f"\n{str(idx)[:40]:<40} [{row['acquisition_tier']}]")
        print(f"  M&A Score: {row['ma_score']:.1f} | Portfolio: {row['portfolio_size']:.0f} games")
        print(f"  Revenue: ${row['total_revenue_usd']:,.0f} | Users: {row['total_user_base']:,.0f}")
        print(f"  Hit Rate: {row['hit_rate']:.1%} | Retention: {row['retention_score']:.1f}")
        print(f"  Risk Score: {row['risk_score']:.2f}")
# ============================================
# 11. REORDER COLUMNS FOR BETTER ORGANIZATION
# ============================================
print("\nREORDERING COLUMNS...")
print("-" * 60)

# Define column order groups
column_order = [
    # Core identifiers
    'app_id',
    'name',
    'type',
    
    # Basic info
    'release_date',
    'game_age_years',
    'is_free',
    
    # Publishers and developers
    'developer',
    'publisher',
    
    # Pricing - original currency
    'currency',
    'price_final_cents',
    'price_initial_cents',
    'price_final',
    'price_initial',
    'price_final_formatted',
    'discount_percent',
    'discount',
    
    # Pricing - USD converted
    'price_final_usd',
    'price_initial_usd',
    'price_avg_usd',
    'price_usd',
    'initial_price_usd',
    
    # User base and engagement
    'owners_range',
    'owners_avg',
    'concurrent_users_yesterday',
    'playtime_average_forever',
    'playtime_average_2weeks',
    'playtime_median_forever',
    'playtime_median_2weeks',
    
    # Revenue and success metrics
    'estimated_revenue_usd',
    'is_successful',
    'is_hit',
    
    # Reviews and ratings
    'review_score',
    'review_score_description',
    'positive',
    'negative',
    'total',
    'total_reviews',
    'positive_ratio',
    'quality_score',
    'metacritic_score',
    'recommendations',
    'steamspy_user_score',
    'steamspy_score_rank',
    'steamspy_positive',
    'steamspy_negative',
    
    # Review text (cleaned versions)
    'reviews_clean',
    
    # Languages
    'languages',
    'languages_clean',
    'languages_full_audio',
    
    # Categories
    'categories_all',
    'category_count',
    'has_multiplayer',
    'has_singleplayer',
    'has_coop',
    'has_vr',
    'has_controller',
    
    # Tags
    'tags_all',
    'tag_count',
    'is_indie',
    'is_action',
    'is_adventure',
    'is_rpg',
    'is_strategy',
    'is_simulation',
    'is_puzzle',
    
    # Genres
    'genres',
    'genres_all',
    'primary_genre',
    'genre_count',
    
    # Descriptions (cleaned versions)
    'summary_clean',
    'summary_length',
    'extensive_clean',
    'extensive_length',
    'about_clean',
    'about_length',
    'short_description_clean',
    'short_description_length',
    'detailed_description_clean',
    'detailed_description_length',
    'about_the_game_clean',
    'about_the_game_length',
    
    # Promotional
    'promo_material_count',
    'promo_types',
]

# Get existing columns
existing_cols = [col for col in column_order if col in merged.columns]

# Get any remaining columns not in the order list
remaining_cols = [col for col in merged.columns if col not in existing_cols]

# Combine ordered and remaining columns
final_column_order = existing_cols + remaining_cols

# Reorder the dataframe
merged = merged[final_column_order]
print(f"Columns reordered: {len(final_column_order)} total")
print(f"   Organized columns: {len(existing_cols)}")
print(f"   Additional columns: {len(remaining_cols)}")




# ============================================
# 12. SAVE CLEANED DATA
# ============================================
print("\nSAVING CLEANED RESULTS")
print("-" * 60)

output_dir = DATA_DIR / "output"
output_dir.mkdir(exist_ok=True)

# Save main dataset
merged_file = output_dir / "steam_data_cleaned_complete.csv"
merged.to_csv(merged_file, index=False)
print(f"Saved cleaned data: {merged_file}")
print(f"   Shape: {merged.shape}")

# Save publisher metrics
if 'publisher_metrics' in locals():
    publisher_file = output_dir / "publisher_metrics_cleaned.csv"
    publisher_metrics.to_csv(publisher_file)
    print(f"Saved publisher metrics: {publisher_file}")

# Save a sampled version (1000 records) of the final dataset
print("\nCreating sampled dataset...")
sampled_merged = merged.sample(n=min(1000, len(merged)), random_state=42)
sampled_file = output_dir / "steam_data_cleaned_sample_1000.csv"
sampled_merged.to_csv(sampled_file, index=False)
print(f"Saved sampled data (1000 records): {sampled_file}")

# Also create a sampled publisher metrics (top 100)
if 'publisher_metrics' in locals():
    sampled_publisher_file = output_dir / "publisher_metrics_top100.csv"
    publisher_metrics.head(100).to_csv(sampled_publisher_file)
    print(f"Saved top 100 publishers: {sampled_publisher_file}")

# Save enhanced data quality report
quality_report = output_dir / "data_quality_report.txt"
with open(quality_report, 'w') as f:
    f.write("DATA QUALITY REPORT - ENHANCED VERSION\n")
    f.write("=" * 60 + "\n\n")
    
    f.write("Dataset Shape:\n")
    f.write(f"  Rows: {len(merged):,}\n")
    f.write(f"  Columns: {len(merged.columns)}\n\n")
    
    f.write("Currency Conversion:\n")
    if 'currency' in merged.columns:
        currency_counts = merged['currency'].value_counts().head(10)
        for curr, count in currency_counts.items():
            f.write(f"  {curr}: {count:,} games\n")
    f.write(f"  Total Revenue (USD): ${merged['estimated_revenue_usd'].sum():,.0f}\n\n")
    
    f.write("Top 20 Columns with Missing Data:\n")
    missing = merged.isnull().sum()
    missing_pct = (missing / len(merged) * 100).round(2)
    missing_sorted = missing.sort_values(ascending=False).head(20)
    for col in missing_sorted.index:
        # Handle potential None values in column name, missing count, and percentage
        col_name = str(col) if col is not None else "UNKNOWN"
        miss_count = int(missing[col]) if pd.notna(missing[col]) else 0
        pct_value = float(missing_pct[col]) if pd.notna(missing_pct[col]) else 0.0
        f.write(f"  {col_name:<35} {miss_count:>7,} ({pct_value:>5.1f}%)\n")
    
    f.write(f"\n\nData Completeness: {(merged.notna().sum().sum() / (len(merged) * len(merged.columns)) * 100):.1f}%\n")
    
    f.write("\n\nAll Column Names ({} total):\n".format(len(merged.columns)))
    for i, col in enumerate(merged.columns, 1):
        col_name = str(col) if col is not None else "UNKNOWN"
        f.write(f"  {i:3}. {col_name}\n")
print(f"Saved data quality report: {quality_report}")

print("\n" + "=" * 80)
print("ENHANCED DATA CLEANING PIPELINE COMPLETE!")
print(f"\nSUMMARY:")
print(f"  • Total games: {len(merged):,}")
print(f"  • Total columns: {len(merged.columns)}")
print(f"  • Publishers analyzed: {publisher_metrics.shape[0] if 'publisher_metrics' in locals() else 0:,}")
print(f"  • Data quality: {(merged.notna().sum().sum() / (len(merged) * len(merged.columns)) * 100):.1f}% complete")
print(f"  • Total revenue (USD): ${merged['estimated_revenue_usd'].sum():,.0f}")
print(f"  • Average tags per game: {merged['tag_count'].mean():.1f}" if 'tag_count' in merged.columns else "")
print("\nOutput files in /output folder:")
print("  • steam_data_cleaned_complete.csv (all data with USD conversion)")
print("  • publisher_metrics_cleaned.csv (USD-based metrics)")
print("  • data_quality_report.txt")
print("=" * 80)

Found Script 6 outputs - using cleaned games data
Data directory: /Users/timtoepper/Downloads/Banks_Assignment_Tim
Games file: /Users/timtoepper/Downloads/Banks_Assignment_Tim/sampled_outputs/games_cleaned.csv

1. LOADING AND CLEANING GAMES.CSV
------------------------------------------------------------
Loaded: 140,082 games from games_cleaned.csv
Cleaned games: 140,082 rows × 17 columns

2. LOADING AND CLEANING STEAMSPY_INSIGHTS.CSV
------------------------------------------------------------
  Processing row 25,000...
  Processing row 50,000...
  Processing row 75,000...
  Processing row 100,000...
  Processing row 125,000...
Loaded: 140,077 records
  Parsed owners - Average: 72,581
Merged steamspy: 140,082 rows × 34 columns

3. LOADING AND CLEANING REVIEWS.CSV
------------------------------------------------------------
Loaded: 140,086 records
  Cleaning reviews text...
  Cleaned reviews text: 11,698 valid reviews
Merged reviews: 140,082 rows × 49 columns

4. LOADING AND CLEANING C